In [1]:
#yolo algorthim
#9 * 9 will be ok for two number and 19 * 19 for multiply number
#evaluate method: intersection over union IoU = overlap/union
#Non-max-Suppression
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Variable
from torchvision import transforms
import pickle

In [2]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.6):
    """box_confidence : (5 * 5 * 6 * 1)
       boxes : (5 * 5 * 6 * 4)
       box_class_probs : (5 * 5 * 6 * 10)"""
    #compute box score
    boxs_score = box_confidence * box_class_probs
    box_class = torch.max(boxs_score, -1)[1]
    box_class_scores = torch.max(boxs_score, -1)[0]
    filter_mask = boxs_score >= threshold
    scores = box_class_scores[filter_mask]
    boxes = boxes[filter_mask]
    classes = box_class[filter_mask]
    return scores, boxes, classes

In [3]:
#load the data
pkl_file = open('two_numbers_dataset_6k.pkl', 'rb')
raw_datasets = pickle.load(pkl_file, encoding = 'bytes')
img = []
label = []
location = []

batch_size = 20

for i in range(len(raw_datasets)):
    img1 = transforms.ToTensor()(raw_datasets[i][0].reshape(80, 80, 1))
    img.append(img1)
    label.append([raw_datasets[i][1][0][0], raw_datasets[i][1][1][0]])
    location.append([raw_datasets[i][1][0][1:], raw_datasets[i][1][1][1:]])

In [4]:
train_len = int(2 * len(raw_datasets)/3)
train_img = img[:train_len]
test_img = img[train_len:len(raw_datasets)]
tr_list_label = label[:train_len]
te_list_label = label[train_len:len(raw_datasets)]
tr_list_location = location[:train_len]
te_list_location = location[train_len:len(raw_datasets)]
train_label = []
test_label = []
train_location = []
test_location = []
for i in range(train_len):
    train_label.append(torch.from_numpy(np.array(tr_list_label[i])))
    train_location.append(torch.from_numpy(np.array(tr_list_location[i])))
for i in range(len(raw_datasets) - train_len):
    test_label.append(torch.from_numpy(np.array(te_list_label[i])))
    test_location.append(torch.from_numpy(np.array(te_list_location[i])))

In [5]:
batch_img = []
batch_label = []
batch_location = []
for i in range(int(train_len/batch_size)):
    batch_img.append(torch.stack(train_img[batch_size * i : batch_size * (i+1)], 0))
    batch_label.append(torch.stack(train_label[batch_size * i : batch_size * (i+1)], 0))
    batch_location.append(torch.stack(train_location[batch_size * i : batch_size * (i + 1)], 0))

In [6]:
# for one small image is 28 * 28, So we need to resize the initial image to: 14 * 14, stride is 4
# module one: use the p as the variable to update, generate 14 * 14 windows to detect
# module two: use the coordinate (x, y) of four bounding boxes to update to localization


"""method one"""
class localization_module(nn.Module):
    def __init__(self):
        super(localization_module, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 5, padding = 2) #80 * 80
        self.conv2 = nn.Conv2d(16, 32, 5, padding = 2) #40 * 40
        self.conv3 = nn.Conv2d(32, 64, 5) #16 * 16 
        self.conv4 = nn.Conv2d(64, 128, 3) #14 * 14
        self.conv5 = nn.Conv2d(128, 1, 1) #14 * 14
    def forward(self, x):
        if torch.cuda.is_available():
            x = x.cuda()
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        return F.sigmoid(x) 
"""method two"""
class regression_local_module(nn.Module):
    def __init__(self):
        super(regression_local_module, self).__init__()
        #extract the feature
        self.conv1 = nn.Conv2d(1, 16, 5, padding = 2) #80 * 80
        self.conv2 = nn.Conv2d(16, 32, 5, padding = 2) #40 * 40
        self.conv3 = nn.Conv2d(32, 64, 5, padding = 2)#20 * 20
        self.conv4 = nn.Conv2d(64, 128, 5, padding = 2)#10 * 10
        self.fc1 = nn.Linear(128 * 5 * 5, 1024)
        self.fc2 = nn.Linear(1024, 8)
    def forward(self, x):
        if torch.cuda.is_available():
            x = x.cuda()
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = F.max_pool2d(F.relu(self.conv4(x)), 2)
        x = x.view(-1, 128 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        return x
"""method three"""
class yolo_like_module(nn.Module):
    def __init__(self):
        super(yolo_local_module, self).__init__()
    def forward(self, x):
        self.conv1 = nn.Conv2d
        
        

In [8]:
class loss(nn.Module):
    def __init__(self):
        super(loss, self).__init__()
        return
    def forward(self, x, grounding_truth, batch_size):
        
        #x is batch_size * 14 * 14
        #grounding_truth is batch_size * 2 * 4
        x = x.data #float tensor
        x = torch.squeeze(x)
        grounding_truth = grounding_truth.data  #long tensor
        grounding_truth = grounding_truth.float()
        x = x.cpu() #move to cpu
        truth1 = torch.index_select(grounding_truth, 1, torch.LongTensor([0])) #batch_size * 1 * 4
        truth2 = torch.index_select(grounding_truth, 1, torch.LongTensor([1])) #batch_size * 1 * 4
        truth1 = torch.index_select(truth1, 2, torch.LongTensor([0, 1])) #batch_size * 1 * 2
        truth2 = torch.index_select(truth2, 2, torch.LongTensor([0, 1])) #batch_size * 1 * 2
        
        coordinate = torch.zeros(batch_size, 14, 14, 2) #batch_size * 14 * 14 * 2
        for i in range(14):
            for j in range(14):
                coordinate[:, i, j, 0] = 14 + 4 * i # x
                coordinate[:, i, j, 1] = 14 + 4 * j # y
        loss_matrix_x = torch.pow((coordinate[:,:,:,0] - torch.unsqueeze(truth1, 2)[:,:,:,0] - 14), 2) #14 * 14
        loss_matrix_y = torch.pow((coordinate[:,:,:,1] - torch.unsqueeze(truth1, 2)[:,:,:,1] - 14), 2) #14 * 14
        loss_matrix1 = loss_matrix_x + loss_matrix_y
        loss_matrix_x = torch.pow((coordinate[:,:,:,0] - torch.unsqueeze(truth2, 2)[:,:,:,0] - 14), 2)
        loss_matrix_y = torch.pow((coordinate[:,:,:,1] - torch.unsqueeze(truth2, 2)[:,:,:,1] - 14), 2)
        loss_matrix2 = loss_matrix_x + loss_matrix_y
        min_loss_matrix = torch.min(loss_matrix1, loss_matrix2) #batch_size * 14 * 14
        min_loss_matrix = min_loss_matrix/(80 * 80) #scaling
        loss1 = torch.sum(x * x * min_loss_matrix, 2) #batch_size * 14
        loss2 = torch.sum(loss1, 1)/(14 * 14)#batch_size
        loss3 = torch.sum(loss2, 0)/batch_size #1
        return (Variable(loss3, requires_grad = True), x)

In [7]:
"""model = localization_module()
if torch.cuda.is_available():
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
train_loss = []
train_accu = []
for epoch in range(10):
    for data, target in zip(batch_img, batch_location):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        target = torch.squeeze(target)
        customize_loss = loss()
        loss1 = customize_loss(output, target, batch_size)[0]
        p = customize_loss(output, target, batch_size)[1]
        loss1.backward()
        train_loss.append(loss1.data[0])
        optimizer.step()
        if i % 80 == 0:
            print('Train Step: {}\tLoss: {:.3f}'.format(i, loss1.data[0]))
        i += 1
"""
def split(grounding_truth):
    truth1 = torch.index_select(grounding_truth, 1, torch.LongTensor([0])) #batch_size * 1 * 4
    truth2 = torch.index_select(grounding_truth, 1, torch.LongTensor([1])) #batch_size * 1 * 4
    truth1 = torch.index_select(truth1, 2, torch.LongTensor([0, 1])) #batch_size * 1 * 2
    truth2 = torch.index_select(truth2, 2, torch.LongTensor([0, 1])) #batch_size * 1 * 2
    return (truth1, truth2)
model = regression_local_module()
if torch.cuda.is_available():
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr = 0.1)
train_loss = []
train_accu = []
func_loss = torch.nn.MSELoss()
for epoch in range(10):
    for data, target in zip(batch_img, batch_location):
        target1, target2 = split(target)
        data, target1, target2 = Variable(data), Variable(target1), Variable(target2)
        if torch.cuda.is_available():
            data = data.cuda()
            target1 = target1.cuda()
            target2 = target2.cuda()
        output = model(data)
        output1, output2, output3, output4 = torch.chunk(output, 4, dim = 1)
        target1, target2 = target1.float(), target2.float()
        loss1 = torch.min(func_loss(output1, target1), func_loss(output1, target2))
        loss2 = torch.min(func_loss(output2, target1), func_loss(output2, target2))
        loss3 = torch.min(func_loss(output3, target1), func_loss(output3, target2))
        loss4 = torch.min(func_loss(output4, target1), func_loss(output4, target2))
        loss = (loss1 + loss2 + loss3 + loss4)/4
        optimizer.zero_grad()
        loss.backward()
        train_loss.append(loss.data[0])
        optimizer.step()
        if i % 80 == 0:
            print('Train Step: {}\tLoss: {:.3f}'.format(i, loss.data[0]))
        i += 1
        

    

Train Step: 240	Loss: 380.574
Train Step: 320	Loss: 302.252
Train Step: 400	Loss: 342.398
Train Step: 480	Loss: 479.451
Train Step: 560	Loss: 518.436
Train Step: 640	Loss: 399.382
Train Step: 720	Loss: 547.723
Train Step: 800	Loss: 338.634
Train Step: 880	Loss: 453.999
Train Step: 960	Loss: 382.660
Train Step: 1040	Loss: 404.042
Train Step: 1120	Loss: 333.525
Train Step: 1200	Loss: 411.434
Train Step: 1280	Loss: 423.732
Train Step: 1360	Loss: 472.094
Train Step: 1440	Loss: 459.501
Train Step: 1520	Loss: 457.656
Train Step: 1600	Loss: 358.412
Train Step: 1680	Loss: 654.057
Train Step: 1760	Loss: 519.568
Train Step: 1840	Loss: 460.227
Train Step: 1920	Loss: 401.980
Train Step: 2000	Loss: 490.608
Train Step: 2080	Loss: 325.284
Train Step: 2160	Loss: 483.619



(0 ,.,.) = 
  1.8399

(1 ,.,.) = 
  0.3246

(2 ,.,.) = 
 -1.8469

(3 ,.,.) = 
 -1.0766

(4 ,.,.) = 
 -1.1818

(5 ,.,.) = 
 -0.5548

(6 ,.,.) = 
  0.2725

(7 ,.,.) = 
  0.2814

(8 ,.,.) = 
 -0.6643

(9 ,.,.) = 
  0.0010

(10,.,.) = 
 -1.5684

(11,.,.) = 
  0.6906

(12,.,.) = 
 -0.0977

(13,.,.) = 
 -0.0578

(14,.,.) = 
  0.1491

(15,.,.) = 
  1.6643

(16,.,.) = 
  1.3831

(17,.,.) = 
  0.1142

(18,.,.) = 
 -1.4542

(19,.,.) = 
  0.4713

(20,.,.) = 
 -1.0370

(21,.,.) = 
  0.7748

(22,.,.) = 
  0.6981

(23,.,.) = 
  0.8950

(24,.,.) = 
  1.8994

(25,.,.) = 
  0.0168

(26,.,.) = 
 -0.9083

(27,.,.) = 
 -0.5421

(28,.,.) = 
 -0.8230

(29,.,.) = 
  0.9957

(30,.,.) = 
  0.3278

(31,.,.) = 
  0.0955

(32,.,.) = 
  1.0895

(33,.,.) = 
 -3.2496

(34,.,.) = 
  0.0649

(35,.,.) = 
 -0.5492

(36,.,.) = 
  0.2020

(37,.,.) = 
  0.1096

(38,.,.) = 
  0.3910

(39,.,.) = 
 -0.0362

(40,.,.) = 
 -0.4080

(41,.,.) = 
 -0.2034

(42,.,.) = 
 -1.7201

(43,.,.) = 
  0.4156

(44,.,.) = 
  0.2415

(45,.,.) 

In [13]:
for i in range(10):
    print(batch_location[i])


(0 ,.,.) = 
   7   5  28  28
  16  39  28  28

(1 ,.,.) = 
  42  26  28  28
  17  40  28  28

(2 ,.,.) = 
  43   9  28  28
  11  16  28  28

(3 ,.,.) = 
  26   3  28  28
   7  16  28  28

(4 ,.,.) = 
  41   8  28  28
  49  41  28  28

(5 ,.,.) = 
  38  19  28  28
  17  24  28  28

(6 ,.,.) = 
   0  16  28  28
  21  45  28  28

(7 ,.,.) = 
  16  33  28  28
  38  36  28  28

(8 ,.,.) = 
  42  41  28  28
  21  42  28  28

(9 ,.,.) = 
  18  12  28  28
  35  37  28  28

(10,.,.) = 
  40  15  28  28
   5  41  28  28

(11,.,.) = 
  21  42  28  28
  40  32  28  28

(12,.,.) = 
  25  32  28  28
  11  16  28  28

(13,.,.) = 
   9   6  28  28
  10  35  28  28

(14,.,.) = 
  12  25  28  28
  31  38  28  28

(15,.,.) = 
   5  13  28  28
   4  34  28  28

(16,.,.) = 
  13  31  28  28
  47  39  28  28

(17,.,.) = 
  34  41  28  28
  34  17  28  28

(18,.,.) = 
  34  22  28  28
  19  51  28  28

(19,.,.) = 
   6  21  28  28
  26  18  28  28

(20,.,.) = 
  12  21  28  28
  32  38  28  28

(21,.,.) = 
